In [ ]:
import pandas as pd
post_priori = pd.read_csv('data/processed/post_priori.csv')

In [2]:
def split_into_seasons(post_priori: pd.DataFrame) -> dict:
    """Split the data into seasons based on predefined dates.
    
    Args:
        post_priori (pd.DataFrame): DataFrame containing post-priori data.
        
    Returns:
        dict: A dictionary containing the sliced dataframes for each season.
    """
    # Define the start and end dates for each season
    season_dates = {
        2009: ('2009-09-10', '2010-01-03'),
        2010: ('2010-09-09', '2011-01-02'),
        2011: ('2011-09-08', '2012-01-01'),
        2012: ('2012-09-05', '2012-12-30'),
        2013: ('2013-09-05', '2013-12-29'),
        2014: ('2014-09-04', '2014-12-28'),
        2015: ('2015-09-10', '2016-01-03'),
        2016: ('2016-09-08', '2017-01-01'),
        2017: ('2017-09-07', '2017-12-31'),
        2018: ('2018-09-06', '2019-01-30')
    }
    
    seasons = {}
    for year, (start_date, end_date) in season_dates.items():
        seasons[year] = post_priori[(post_priori['game_date'] >= start_date) & (post_priori['game_date'] <= end_date)]
    
    return seasons

In [3]:
seasons = split_into_seasons(post_priori)
for year, season in seasons.items():
    print(f'Season {year}: {season.shape[0]} games')

Season 2009: 255 games
Season 2010: 256 games
Season 2011: 252 games
Season 2012: 254 games
Season 2013: 251 games
Season 2014: 256 games
Season 2015: 255 games
Season 2016: 254 games
Season 2017: 255 games
Season 2018: 222 games


In [8]:
seasons[2018].head()


,game_id,game_date,home_team,away_team,result,point_diff,score_home,score_q1_home,score_q1_allow_home,score_q2_home,...,off_tot_pass_tds_away,off_tot_pass_yds_away,off_pass_cmp_perc_away,def_tackles_away,def_sacks_away,def_interceptions_away,def_forced_fumbles_away,def_fumble_recoveries_away,def_defense_qbhit_away,def_defense_safety_away
2288,2018090600,2018-09-06,PHI,ATL,home_win,6,18,0,3,3,...,0.0,225.0,45.652174,39,2.0,1.0,0.0,0.0,4.0,0.0
2289,2018090900,2018-09-09,BAL,BUF,home_win,44,47,14,0,26,...,0.0,70.0,28.205128,48,2.0,0.0,1.0,0.0,4.0,0.0
2290,2018090907,2018-09-09,NYG,JAX,away_win,-4,15,3,3,6,...,1.0,168.0,54.545455,33,2.0,1.0,0.0,0.0,6.0,0.0
2291,2018090906,2018-09-09,NO,TB,away_win,-7,40,10,14,24,...,4.0,417.0,77.777778,45,1.0,0.0,2.0,0.0,5.0,0.0
2292,2018090905,2018-09-09,NE,HOU,home_win,7,27,7,3,21,...,1.0,158.0,45.945946,47,2.0,1.0,2.0,0.0,5.0,0.0


In [194]:
import pandas as pd

def calculate_team_averages(past_games: pd.DataFrame, team: str, game_date: pd.Timestamp, columns_avg: list) -> pd.Series:
    """
    Calculate averages for a team considering both home and away games up to a given date.
    
    Args:
        past_games (pd.DataFrame): DataFrame containing data for a past_games of the current season.
        team (str): Team name.
        game_date (pd.Timestamp): Date of the game for which averages are calculated.
        columns (list): List of columns to calculate averages for.
    Returns:
        pd.Series: A series containing the calculated averages.
    """
    #print(columns_avg)
    if past_games.empty:
        return pd.Series([0] * len(columns_avg), index=columns_avg).to_frame().T      
    
    past_home_games = past_games[past_games['home_team'] == team]
    if past_home_games.empty:
        past_home_games_avg = pd.Series([0] * len(columns_avg), index=columns_avg)
    else:
        past_home_games_avg = past_home_games.filter(like = '_home').mean(numeric_only=True)
        past_home_games_avg.index = columns_avg
        
    past_away_games = past_games[past_games['away_team'] == team]
    if past_away_games.empty:
        past_away_games_avg = pd.Series([0] * len(columns_avg), index=columns_avg)
    else:
        past_away_games_avg = past_away_games.filter(like = '_away').mean(numeric_only=True)
        past_away_games_avg.index = columns_avg   

    
    combined_avg = (past_home_games_avg + past_away_games_avg) 
    combined_avg = combined_avg.to_frame().T
    
    # print(past_home_games_avg)
    # print(past_away_games_avg)
    # print(combined_avg)
    # print('---------------------')
    return combined_avg

def calculate_current_season_averages(post_priori: pd.DataFrame) -> pd.DataFrame:
    """Calculate current season averages for each game.
    
    Args:
        post_priori (pd.DataFrame): DataFrame containing post-priori data.
        
        Returns:
        pd.DataFrame: DataFrame containing the calculated averages for each game.
    """
    post_priori['game_date'] = pd.to_datetime(post_priori['game_date'])
    post_priori = post_priori.sort_values('game_date')
    post_priori.fillna(0, inplace=True)
    
    # Assuming split_into_seasons is defined elsewhere
    seasons = split_into_seasons(post_priori)
    columns = post_priori.filter(like='_home').columns
    columns = [column.replace('_home', '') for column in columns]
    current_season_averages = pd.DataFrame()
    
    for season, season_data in seasons.items():
        season_data = season_data.sort_values('game_date')
        # season_data = season_data[season_data['home_team'] == 'PIT']
        #print(season_data)
        #print(season_data.select_dtypes(include=['number']).columns)
        for index, game in season_data.iterrows():
            game_id = game['game_id']
            game_date = game['game_date']
            home_team = game['home_team']
            away_team = game['away_team']
            result = game['result']
            
            past_games = season_data[season_data['game_date'] < game_date]
            
            
            # Calculate team averages up to this game date
            home_avg_combined = calculate_team_averages(past_games, home_team, game_date, columns)
            away_avg_combined = calculate_team_averages(past_games, away_team, game_date, columns)
            
            # Create a DataFrame with the game details and calculated averages
            game_averages = pd.DataFrame({
                'game_id': [game_id],
                'game_date': [game_date],
                'season': [season],
                'home_team': [home_team],
                'away_team': [away_team],
                'result': [result]
            })
            
            game_averages = pd.concat([game_averages.reset_index(drop=True), home_avg_combined.add_prefix('current_season_avg_home_').reset_index(drop=True), away_avg_combined.add_prefix('current_season_avg_away_').reset_index(drop=True)], axis=1)
            # print(home_avg_combined)
            # print(away_avg_combined)
            # print(game_averages)
            # print('---------------------')
            current_season_averages = pd.concat([current_season_averages, game_averages], ignore_index=True)
    return current_season_averages


In [ ]:
current_season_averages = calculate_current_season_averages(post_priori)

In [200]:
current_season_averages.to_csv('data/processed/averages/current_season_averages.csv', index=False)

In [ ]:
def calculate_mutual_game_averages(historical_data: pd.DataFrame, home_team: str, away_team: str, game_date: pd.Timestamp, columns_avg: list) -> pd.DataFrame:
    """
    Calculate averages for mutual games between two teams up to a given date considering all historical data.
    
    Args:
        historical_data (pd.DataFrame): DataFrame containing data for all historical games.
        home_team (str): Home team name.
        away_team (str): Away team name.
        game_date (pd.Timestamp): Date of the game for which averages are calculated.
        columns_avg (list): List of columns to calculate averages for.
        
    Returns:
        pd.DataFrame: A DataFrame containing the calculated averages for both teams in mutual games.
    """
    # Filter mutual games up to the current game date
    mutual_games = historical_data[((historical_data['home_team'] == home_team) & 
                                    (historical_data['away_team'] == away_team)) |
                                    ((historical_data['home_team'] == away_team) & 
                                    (historical_data['away_team'] == home_team)) &
                                    (historical_data['game_date'] < game_date)]
    
    # Calculate home and away team averages in mutual games
    home_mutual_avg = calculate_team_averages(mutual_games, home_team, game_date, columns_avg)
    away_mutual_avg = calculate_team_averages(mutual_games, away_team, game_date, columns_avg)
    
    return home_mutual_avg.add_prefix('mutual_game_home_'), away_mutual_avg.add_prefix('mutual_game_away_')

In [199]:
mutual_avgs

(   mutual_game_home_score  mutual_game_home_score_q1  \
 0                    19.0                        0.0   
 
    mutual_game_home_score_q1_allow  mutual_game_home_score_q2  \
 0                              3.0                       13.0   
 
    mutual_game_home_score_q2_allow  mutual_game_home_score_q3  \
 0                             10.0                       13.0   
 
    mutual_game_home_score_q3_allow  mutual_game_home_score_q4  \
 0                             28.0                       19.0   
 
    mutual_game_home_score_q4_allow  mutual_game_home_score_q5  ...  \
 0                              7.0                        NaN  ...   
 
    mutual_game_home_off_tot_pass_tds  mutual_game_home_off_tot_pass_yds  \
 0                                0.0                              429.0   
 
    mutual_game_home_off_pass_cmp_perc  mutual_game_home_def_tackles  \
 0                               110.0                          87.0   
 
    mutual_game_home_def_sacks  mutual

In [ ]:
def calculate_mutual_game_averages(historical_data: pd.DataFrame, home_team: str, away_team: str, game_date: pd.Timestamp, columns_avg: list) -> pd.DataFrame:
    """
    Calculate averages for mutual games between two teams up to a given date considering all historical data.
    
    Args:
        historical_data (pd.DataFrame): DataFrame containing data for all historical games.
        home_team (str): Home team name.
        away_team (str): Away team name.
        game_date (pd.Timestamp): Date of the game for which averages are calculated.
        columns_avg (list): List of columns to calculate averages for.
        
    Returns:
        pd.DataFrame: A DataFrame containing the calculated averages for both teams in mutual games.
    """
    # Filter mutual games up to the current game date
    mutual_games = historical_data[((historical_data['home_team'] == home_team) & 
                                    (historical_data['away_team'] == away_team)) |
                                    ((historical_data['home_team'] == away_team) & 
                                    (historical_data['away_team'] == home_team)) &
                                    (historical_data['game_date'] < game_date)]
    
    # If no mutual games exist, return zeros
    if mutual_games.empty:
        return pd.DataFrame({col: [0] * 2 for col in columns_avg}, index=['mutual_game_home', 'mutual_game_away'])
    
    # Calculate home and away team averages in mutual games
    home_mutual_avg = calculate_team_averages(mutual_games, home_team, game_date, columns_avg)
    away_mutual_avg = calculate_team_averages(mutual_games, away_team, game_date, columns_avg)
    
    # Combine and return as a DataFrame
    combined_mutual_avgs = pd.concat([home_mutual_avg.add_prefix('mutual_game_home_'), 
                                    away_mutual_avg.add_prefix('mutual_game_away_')], axis=1)
    
    return combined_mutual_avgs

# Example usage with current season average calculation function
def calculate_current_season_and_mutual_game_averages(post_priori: pd.DataFrame, historical_data: pd.DataFrame) -> pd.DataFrame:
    """Calculate current season and mutual game averages for each game."""
    
    post_priori['game_date'] = pd.to_datetime(post_priori['game_date'])
    post_priori.sort_values('game_date', inplace=True)
    
    # Assuming split_into_seasons is defined elsewhere
    seasons = split_into_seasons(post_priori)
    
    # Get column names without suffixes
    columns = post_priori.filter(like='_home').columns
    columns = [column.replace('_home', '') for column in columns]
    
    current_season_and_mutual_game_averages = pd.DataFrame()
    
    for season, season_data in seasons.items():
        
        for index, game in season_data.iterrows():
            game_id = game['game_id']
            game_date = game['game_date']
            home_team = game['home_team']
            away_team = game['away_team']
            result = game['result']
            
            # Filter out all previous games before this one
            past_games = season_data[season_data['game_date'] < game_date]
            
            # Calculate current season average
            home_current_szn_average = calculate_team_averages(past_games, home_team, game_date, columns)
            away_current_szn_average = calculate_team_averages(past_games, away_team, game_date, columns)
            
            # Calculate mutual games average from historical data
            mutual_game_home_average, mutual_game_away_average = calculate_mutual_game_averages(historical_data,
                                                                                                home_team,
                                                                                                away_team,
                                                                                                game_date,
                                                                                                columns)

            # Create a DataFrame with all calculated averages and append it to result DataFrame
            combined_df_row = {
                'game_id': [game_id],
                'game_date': [game_date],
                'season': [season],
                'home_team': [home_team],
                'away_team': [away_team],
                'result': [result]
            }
            
            combined_df_row.update(home_current_szn_average.add_prefix('current_season_home').to_dict())
            combined_df_row.update(away_current_szn_average.add_prefix('current_season_away').to_dict())
            
            combined_df_row.update(mutual_game_home_average.to_dict())
            combined_df_row.update(mutual_game_away_average.to_dict())

            combined_df_row_df = pd.DataFrame(combined_df_row)

            current_season_and_mutual_game_averages = pd.concat([current_season_and_mutual_game_averages,
                                                                combined_df_row_df], ignore_index=True)

            
# Example usage would be calling `calculate_current_season_and_mutual_game_averages()` with your `post_priori` and `historical_data`.

In [205]:
import pandas as pd

def calculate_team_wins(past_games: pd.DataFrame, team: str, game_date: pd.Timestamp) -> int:
    """
    Calculate the number of wins for a team up to a given date.
    
    Args:
        past_games (pd.DataFrame): DataFrame containing data for past games.
        team (str): Team name.
        game_date (pd.Timestamp): Date of the game for which wins are calculated.
        
    Returns:
        int: Number of wins for the team up to the given date.
    """
    # Filter past games up to the current game date
    past_games = past_games[past_games['game_date'] < game_date]
    
    # Count wins as home team
    home_wins = len(past_games[(past_games['home_team'] == team) & (past_games['result'] == 'home_win')])
    
    # Count wins as away team
    away_wins = len(past_games[(past_games['away_team'] == team) & (past_games['result'] == 'away_win')])
    
    # Total wins
    total_wins = home_wins + away_wins
    
    return total_wins

def calculate_last_n_games_averages(historical_data: pd.DataFrame, team: str, game_date: pd.Timestamp, columns_avg: list, n: int) -> pd.Series:
    """
    Calculate averages based on the last N games played by a team up to a given date.
    
    Args:
        historical_data (pd.DataFrame): DataFrame containing data for all historical games.
        team (str): Team name.
        game_date (pd.Timestamp): Date of the game for which averages are calculated.
        columns_avg (list): List of columns to calculate averages for.
        n (int): Number of last games to consider.
        
    Returns:
        pd.Series: A series containing the calculated averages based on the last N games and number of wins.
    """
    # Filter all past games up to the current game date
    past_games = historical_data[(historical_data['game_date'] < game_date) & 
                                 ((historical_data['home_team'] == team) | (historical_data['away_team'] == team))]
    
    # Get only the last N games
    last_n_games = past_games.tail(n)
    
    # If no past games are available, return zeros for all columns
    if last_n_games.empty:
        return pd.Series([0] * len(columns_avg), index=columns_avg).to_frame().T
    
    # Calculate averages for home and away games separately
    home_games = last_n_games[last_n_games['home_team'] == team]
    away_games = last_n_games[last_n_games['away_team'] == team]
    
    home_avg = home_games.filter(like='_home').mean(numeric_only=True)
    home_avg.index = columns_avg
    
    away_avg = away_games.filter(like='_away').mean(numeric_only=True)
    away_avg.index = columns_avg
    
    # Combine home and away averages
    combined_avg = (home_avg + away_avg)
    
    # Calculate number of wins in the last N games
    total_wins = calculate_team_wins(last_n_games, team, game_date)
    
    # Add number of wins to the result
    combined_avg['number_of_wins'] = total_wins
    
    return combined_avg.to_frame().T

# Example usage
def calculate_last_n_games_for_all(post_priori: pd.DataFrame, n: int) -> pd.DataFrame:
    """Calculate last N games averages and number of wins for each game."""
    
    post_priori['game_date'] = pd.to_datetime(post_priori['game_date'])
    post_priori.sort_values('game_date', inplace=True)
    
    # Get column names without suffixes
    columns = post_priori.filter(like='_home').columns
    columns = [column.replace('_home', '') for column in columns]
    
    last_n_game_averages_with_wins = pd.DataFrame()
    
    for index, game in post_priori.iterrows():
        game_id = game['game_id']
        game_date = game['game_date']
        home_team = game['home_team']
        away_team = game['away_team']
        result = game['result']
        
        # Calculate last N games average from historical data and number of wins
        home_last_n_average_with_wins = calculate_last_n_games_averages(post_priori, home_team, game_date, columns, n)
        away_last_n_average_with_wins = calculate_last_n_games_averages(post_priori, away_team, game_date, columns, n)

        # Convert Series to dictionary and merge with other scalar values
        combined_df_row = {
            'game_id': [game_id],
            'game_date': [game_date],
            'home_team': [home_team],
            'away_team': [away_team],
            'result': [result]
        }
        
        # Convert Series to dictionary and update combined_df_row with it
        combined_df_row.update(home_last_n_average_with_wins.add_prefix(f'last_{n}_games_home_').iloc[0].to_dict())
        combined_df_row.update(away_last_n_average_with_wins.add_prefix(f'last_{n}_games_away_').iloc[0].to_dict())

        # Create a DataFrame from the combined dictionary
        combined_df_row_df = pd.DataFrame(combined_df_row)

        # Append to final result DataFrame
        last_n_game_averages_with_wins = pd.concat([last_n_game_averages_with_wins,
                                                    combined_df_row_df], ignore_index=True)

    return last_n_game_averages_with_wins

# Example usage would be calling `calculate_last_n_games_for_all()` with your `post_priori` data.

In [206]:
last_10_games = calculate_last_n_games_for_all(post_priori, 10)

In [208]:
last_10_games.tail()

,game_id,game_date,home_team,away_team,result,last_10_games_home_score,last_10_games_home_score_q1,last_10_games_home_score_q1_allow,last_10_games_home_score_q2,last_10_games_home_score_q2_allow,...,last_10_games_away_off_pass_cmp_perc,last_10_games_away_def_tackles,last_10_games_away_def_sacks,last_10_games_away_def_interceptions,last_10_games_away_def_forced_fumbles,last_10_games_away_def_fumble_recoveries,last_10_games_away_def_defense_qbhit,last_10_games_away_def_defense_safety,last_10_games_home_number_of_wins,last_10_games_away_number_of_wins
2505,2018121605,2018-12-16,IND,DAL,home_win,57.800000,10.400000,10.000000,29.40,14.600000,...,120.166814,75.666667,4.583333,1.333333,1.416667,0.0,12.083333,0.0,6.0,6.0
2506,2018121609,2018-12-16,SF,SEA,home_win,40.200000,9.600000,11.200000,23.40,13.200000,...,126.120278,83.600000,5.400000,1.000000,2.400000,0.0,12.600000,0.0,2.0,7.0
2507,2018121608,2018-12-16,NYG,TEN,away_win,49.800000,9.600000,7.600000,23.40,15.800000,...,122.595939,82.600000,5.600000,0.800000,1.800000,0.0,11.200000,0.2,4.0,5.0
2508,2018121603,2018-12-16,CHI,GB,home_win,58.166667,10.083333,4.916667,30.25,7.500000,...,113.275936,94.200000,8.000000,1.200000,1.800000,0.0,12.800000,0.0,7.0,4.0
2509,2018121700,2018-12-17,CAR,NO,away_win,53.416667,12.416667,12.166667,31.25,15.666667,...,142.646542,88.666667,7.500000,2.500000,2.666667,0.0,14.000000,0.0,4.0,9.0


In [209]:
last_10_games.to_csv('data/processed/averages/last_10_games.csv', index=False)